In [1]:

from gpt_utility import mkTrainingBatch_ft, get_testing_list, save_file_response_as_jsonl, read_jsonl_file, parse_gpt_result, client,read_csv
model = '[Your finetuned gpt model]'
testing_index_list = get_testing_list('data/test_indices.txt')
testing_index_list2 = get_testing_list('data/test_indices_2.txt')
mkTrainingBatch_ft(testing_index_list,model,'output/ft_test')

finish writing batch file: output/ft_test.jsonl


In [ ]:
#Submit batched input to gpt api.
batch_input_file = client.files.create(
  file=open("output/ft_test.jsonl", "rb"),
  purpose="batch"
)
batch_input_file_id = batch_input_file.id

client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "nightly eval job"
    })
#You will recieve a Batch object from OpenAI, note the content in field 'id'

In [ ]:
client.batches.retrieve('[ID_HERE]') #[ID_HERE]
#Call to check the status of data processing, once status='completed', note the content in output_file_id field.

In [ ]:
file_response = client.files.content('[OUTPUT_FILE_ID_HERE]') #"[OUTPUT_FILE_ID_HERE]"

In [ ]:
save_file_response_as_jsonl(file_response, 'output/batch_request_ft.jsonl')
prompted_result = read_jsonl_file('output/batch_request_ft.jsonl')
parse_gpt_result(prompted_result, 'output/pii_ft_detected.csv')


response_all = read_csv('output/batch_response.csv')
filtered_df = response_all[response_all['file_idx'].isin(testing_index_list2)]
filtered_df.to_csv('output/pii_ft_detected_2.csv', index=False)